### Import Required Package

In [1]:
import pandas as pd
import talib
import numpy as np
import json
import os
os.chdir('/home/jovyan/individualized-indicator')
import yaml
import glob
import pandas as pd
import gcsfs
fs = gcsfs.GCSFileSystem(project="dst-dev2021")
from backtest.simulated_backtest_pkl import *
from datetime import datetime
from dateutil.relativedelta import relativedelta

/opt/conda/envs/py310/lib/python3.10/site-packages/pyfolio/pos.py:25: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


### Import Tidal

In [2]:
import tidal as td

### Initialize Plumber

In [3]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

In [4]:
today = datetime.today()
one_month_ago = today - relativedelta(months=1)
formatted_date = one_month_ago.strftime('%Y-%m-%d')

total_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": formatted_date,
        "tickers": ",".join([stock for stock in ['Y9999']]),
    },
)

latest_data = total_data.reset_index()
latest_data = latest_data.iloc[-1][1].strftime("%Y-%m-%d")
latest_data
total_data

open      high       low     close      volume
ticker datetime                                                      
Y9999  2024-10-28  23455.69  23547.12  23164.42  23198.07  6484900000
       2024-10-29  23048.45  23056.57  22711.10  22926.59  7002228000
       2024-10-30  22985.13  23116.33  22801.45  22820.43  6385074000
       2024-11-01  22438.93  22828.98  22231.94  22780.08  8015830000
       2024-11-04  22839.18  23020.99  22792.77  22965.39  6638863000
       2024-11-05  22870.11  23178.80  22849.83  23106.79  6504125000
       2024-11-06  23112.95  23439.39  23035.30  23217.38  8013368000
       2024-11-07  23180.01  23518.63  23132.94  23408.82  8714755000
       2024-11-08  23608.11  23699.70  23509.13  23553.89  8522966000
       2024-11-11  23477.75  23563.80  23306.66  23529.64  7157746000
       2024-11-12  23276.76  23276.76  22981.77  22981.77  8798515000
       2024-11-13  22894.91  23046.47  22860.23  22860.23  7454299000
       2024-11-14  22821.04  22893.82  22619.71  22715.38  8159633000
       2024-11-15  22767.11  22940.80  22690.77  22742.77  7887946000
       2024-11-18  22693.96  22781.77  22503.69  22546.54  7378710000
       2024-11-19  22614.74  22906.69  22578.75  22848.80  7496707000
       2024-11-20  22872.91  22977.56  22622.47  22688.36  7304592000
       2024-11-21  22671.66  22708.57  22490.28  22555.66  6888868000
       2024-11-22  22799.53  23019.04  22799.53  22904.32  7592870000
       2024-11-25  22996.05  23172.43  22948.37  22948.37  9016499000
       2024-11-26  22789.60  22832.11  22645.13  22678.76  6587493000
       2024-11-27  22665.58  22743.99  22334.78  22334.78  7465837000

### Simulated_price

In [5]:
ind_start = '2021-11-01'
ind_end = latest_data
# ind_end = datetime.today().strftime('%Y-%m-%d')
day_start = '2024-04-01'
day_end = '2024-08-30'
test_start = '2024-09-02'
# 取得當天的日期
test_end = ind_end
train_season = '2024_06_01'
emb_length = 'embeddings_length200_2024_06_01.npy'

etl = recommend_stock(emb_length, ind_start, ind_end, day_start, day_end, test_start, test_end, train_season)

indicator_path_r = 'jeff-stock-wise/indicator/'+ train_season + '/indicator_chose.csv'
        
with fs.open(indicator_path_r, 'r') as f:
    indicator_top_list = pd.read_csv(f, index_col=0)

TOP_k = len(indicator_top_list)

# 回測
input_backtest_table = Input_backtest_table(etl,'top_'+str(TOP_k), test_start, test_end, train_season)
input_backtest_table_calculate = input_backtest_table.calculate()

ok


100%|██████████| 324/324 [00:59<00:00,  5.42it/s]


### Load market data and compute technical indicators

In [6]:
gcs_path = 'jeff-stock-wise/simulated/'

start_date = "2024-10-01"
end_date = latest_data

# 初始化一個空的字典來存儲所有讀取的 DataFrame
dataframes = {}

# 遍歷目錄中的所有檔案
for filename in fs.ls(gcs_path):
    if filename.endswith('.pkl'):
        # 組合完整的檔案路徑
        with fs.open(filename, 'rb') as f:
        # 讀取 .pkl 檔案
            df = pd.read_pickle(f)
        
        # 使用檔案名稱（不包括副檔名）作為字典的鍵
        variable_name = filename.split('_')[2]
        
        # 將 DataFrame 儲存到字典中
        dataframes[variable_name] = df

keys_list = list(dataframes.keys())
# 將字符串轉換為 datetime 格式，然後進行排序
sorted_dates = sorted(pd.to_datetime(keys_list))
keys_list = [date.strftime('%Y-%m-%d') for date in sorted_dates]

total_df = pd.DataFrame()

for j in range(0,len(keys_list)):
    df_name = keys_list[j]
    selected_df = dataframes[df_name]    
    selected_df = selected_df.reset_index()
    stock_list = pd.DataFrame(selected_df.iloc[:,0].unique())
    total_df = pd.concat([total_df, stock_list], ignore_index=True)
    
unique_values_list = list(total_df.iloc[:,0].unique())
sorted_list = sorted(unique_values_list)

total_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in sorted_list]),
    },
)

total_data.index = total_data.index.set_levels(
    pd.to_datetime(total_data.index.levels[1]),level=1)

total_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

total_data['new'] = -1
total_data.reset_index(inplace=True)

def filter_by_year_month_12(filter_data, select_year, select_mon):
    # 計算當前篩選年份和下一個年份
    next_year = select_year + 1
    
    # 構建篩選條件：當前年12月、次年1月、次年2月
    filter_data = filter_data[
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 1)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 2)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 3))
    ]
    
    return filter_data

concat_df = pd.DataFrame()

# for i in range(0,len(keys_list)-1):
for i in range(0,len(keys_list)):
    df_name = keys_list[i]
    selected_df = dataframes[df_name]
    selected_df.reset_index(inplace=True)
    stock_df = selected_df.iloc[:,0]
    stock_df_unique = list(stock_df.unique())
    result = list(set(sorted_list) - set(stock_df_unique))
    filtered_df = total_data[total_data['instrument'].isin(result)]
    filtered_df = filtered_df.rename(columns={'volume': 'vol'})
    selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
    
    select_year = int(df_name.split('-')[0])
    select_mon = int(df_name.split('-')[1])
    select_mons = [(select_mon+1)%12, (select_mon+2)%12, (select_mon+3)%12]
    select_mons = [12 if mon == 0 else mon for mon in select_mons]
    
    selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
    # 過濾出所有6月的交易資料
    filter_data = selected_df[selected_df['datetime'].dt.month.isin(select_mons)]

    # 使用時，根據 select_year 和 select_mon 動態篩選數據
        
    if select_mon == 12:
        filter_data = filter_by_year_month_12(filter_data, select_year, select_mon) 

    else:
        filter_data = filter_data[filter_data['datetime'].dt.year == select_year]
    
    concat_df = pd.concat([concat_df, filter_data], ignore_index=True)
    
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

concat_df.sort_values(by=['instrument', 'datetime'], inplace=True)
concat_df.reset_index(drop=True, inplace=True)
row = np.where(concat_df.iloc[:,0] == '6251')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '8406')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '6548')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
quote_data = concat_df.set_index(['instrument', 'datetime'])
quote_data.columns.values[4] = 'volume'
quote_data
# quote_data.to_csv('./quote_data.csv')

open    high     low   close      volume       new
instrument datetime                                                        
1101       2024-10-01   33.90   33.90   33.45   33.70  15831917.0  0.004736
           2024-10-04   33.50   34.00   33.50   33.80  17712471.0  0.005326
           2024-10-07   33.80   33.85   33.30   33.50  13191324.0  0.005014
           2024-10-08   33.25   33.45   32.90   33.05  14236524.0  0.005528
           2024-10-09   32.90   32.95   32.45   32.45  11396786.0  0.005019
...                       ...     ...     ...     ...         ...       ...
9958       2024-11-21  176.00  182.00  176.00  181.00   2625672.0  0.000846
           2024-11-22  182.00  184.50  180.00  180.50   2093628.0  0.000649
           2024-11-25  182.00  185.00  182.00  184.50   1605437.0  0.000888
           2024-11-26  184.50  185.00  182.00  182.50   1297594.0  0.000745
           2024-11-27  184.50  186.00  179.50  179.50   2324523.0  0.000691

[12312 rows x 6 columns]

### Strategy Lake API

In [7]:
data_api_url = ("https://dev-api.ddt-dst.cc/api/plumber/nextron/industry_prod_df")
df = pd.read_parquet(
    data_api_url,
    storage_options={
        "gcp-token": json.dumps(token),
    }
)

### Load benchmark data

In [8]:
benchmark_inst = "0050"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
    inplace=True,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

/tmp/ipykernel_4684/2173060872.py:11: FutureWarning: inplace is deprecated and will be removed in a future version.
  benchmark_data.index.set_levels(


open    high     low   close    volume
instrument datetime                                            
0050       2024-10-01  183.90  184.60  183.35  183.60   9883064
           2024-10-04  184.00  185.30  183.00  183.60   9244059
           2024-10-07  185.80  188.20  185.80  188.15  13993911
           2024-10-08  186.95  187.35  185.50  187.25   9953432
           2024-10-09  189.00  190.20  189.00  189.00  11860866
           2024-10-11  189.05  192.80  189.05  192.15  13197709
           2024-10-14  192.10  192.90  190.95  192.30  10725917
           2024-10-15  193.00  196.60  193.00  196.40  15738243
           2024-10-16  192.45  195.30  191.65  193.00  18604770
           2024-10-17  194.00  194.10  191.90  193.30   9936801
           2024-10-18  199.30  200.20  197.25  198.10  33596096
           2024-10-21  199.05  199.95  198.00  198.05  13716495
           2024-10-22  197.20  197.30  195.35  197.30   8995727
           2024-10-23  196.70  196.90  195.65  196.00  10050673
           2024-10-24  196.00  196.70  194.50  194.80   8642395
           2024-10-25  195.00  196.55  195.00  196.40  12961246
           2024-10-28  197.90  198.15  194.95  196.05  11929580
           2024-10-29  193.25  193.25  190.20  191.50  26008937
           2024-10-30  192.95  194.05  191.50  191.80  14531256
           2024-11-01  186.15  190.55  185.95  190.50  21072030
           2024-11-04  191.05  193.20  190.80  192.95   7681874
           2024-11-05  191.55  194.70  191.40  193.65  10565332
           2024-11-06  194.05  198.05  193.50  195.20  17354391
           2024-11-07  194.90  198.05  194.50  197.45  13976813
           2024-11-08  199.40  200.00  198.70  199.00  13571143
           2024-11-11  198.85  199.10  196.80  199.10  12949097
           2024-11-12  195.20  195.85  194.00  194.00  28057434
           2024-11-13  193.85  194.00  192.50  192.75   9953901
           2024-11-14  192.30  192.50  190.10  191.00  13318035
           2024-11-15  192.05  192.85  190.85  192.35   7809314
           2024-11-18  191.45  191.70  189.60  189.65  14637016
           2024-11-19  190.70  192.95  190.20  192.25   7932841
           2024-11-20  192.80  193.35  190.75  191.45  12509186
           2024-11-21  190.00  190.15  188.60  189.35  17071862
           2024-11-22  191.10  193.20  191.10  193.05   8291407
           2024-11-25  194.70  194.75  192.25  192.35   8215118
           2024-11-26  190.10  190.30  188.80  189.85  12427002
           2024-11-27  189.15  189.45  187.00  187.10  15890418

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [9]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=50000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True, reqMem="1000Mi", ignore_volume_size=True)

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(10, 1, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

[2024/11/28 10:18:32] root INFO Tidal client version: 1.1.41
[2024/11/28 10:18:32] root ERROR SocketClient tcp://10.136.7.2:6666: No module named 'tidal'
[2024/11/28 10:18:32] root INFO Tidal server version: None


### Stock Config

In [10]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [11]:
tidal.exchange.inst_configs

{'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0}}

### Config Modification

In [12]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [13]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [14]:
tidal.backtest()
logs = tidal.get_logs()
for log in logs:
    print(log)

Tidal Backtesting: 100%|██████████| 38/38 [00:04<00:00,  9.02it/s, cash=1.18e+5, pnl=-8.35e+5, position_cost=4.7e+7, value=4.63e+7] 


### Metric - AccountInfo

In [15]:
account_info = tidal.metrics["AccountInfo"].report
print(account_info)

                    cash     trade_cost  position_cost  slip_cost        pnl  \
datetime                                                                       
2024-10-01  5.000000e+07       0.000000            0.0        0.0        0.0   
2024-10-04  3.037287e+06   15386.986162     46947326.2    48576.2  -803326.2   
2024-10-07  3.864301e+05   32168.082685     49222275.4    46251.8  -205475.4   
2024-10-08  5.594590e+05   49015.206841     48941615.2   -84524.2  -454065.2   
2024-10-09  2.720459e+05   66286.715940     49283096.8  -106382.6  -882046.8   
2024-10-11  1.736792e+05   82776.004121     49432595.2   -55505.2  -679345.2   
2024-10-14  2.688874e+05  109655.246968     49686001.0    44307.4  -473101.0   
2024-10-15  2.175627e+05  126710.817674     49689026.1    67076.5  -643326.1   
2024-10-16  3.617232e+05  143950.334684     49628906.1    51176.5  -544706.1   
2024-10-17  4.739817e+05  160884.839190     49446981.0    81483.5  -786331.0   
2024-10-18  2.325276e+05  178785.349249 

### Metric - PositionInfo

In [16]:
position_df = tidal.metrics["PositionInfo"].report
print(position_df)

                       quantity    price   commission  slip_cost       pnl  \
instrument datetime                                                          
1301       2024-10-04   89000.0   53.106  1549.088744     9434.0   -9434.0   
           2024-10-07   89000.0   53.106  1549.088744     9434.0  -80634.0   
           2024-10-08   89000.0   53.106  1549.088744     9434.0 -205234.0   
           2024-10-09   89000.0   53.106  1549.088744     9434.0 -316484.0   
           2024-10-11   89000.0   53.106  1549.088744     9434.0 -312034.0   
...                         ...      ...          ...        ...       ...   
8016       2024-11-19   19000.0  217.434  1354.015877   -20254.0  -27246.0   
           2024-11-20   19000.0  217.434  1354.015877   -20254.0  -93746.0   
           2024-11-22   10000.0  215.430   706.071825     4300.0   -4300.0   
           2024-11-25   10000.0  215.430   706.071825     4300.0   -9300.0   
           2024-11-26   10000.0  215.430   706.071825     4300.0

In [17]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl,now_price
instrument,datetime,,,,,,


### Metric - AdditionalInfo

In [18]:
tidal.metrics['AdditionalInfo'].report

new
instrument datetime            
1101       2024-10-01  0.004736
           2024-10-04  0.005326
           2024-10-07  0.005014
           2024-10-08  0.005528
           2024-10-09  0.005019
...                         ...
9958       2024-11-21  0.000846
           2024-11-22  0.000649
           2024-11-25  0.000888
           2024-11-26  0.000745
           2024-11-27  0.000691

[12312 rows x 1 columns]

### Strategy Lake Submit

In [19]:
# group = dict(zip(df["coid"].astype(str), df["tejind4_c"]))
# submit_lake_backtest_result = tidal.submit_lake_monitor(account_info=account_info, position_df=position_df, benchmark_info=benchmark_data, group=group,lake_env="dev",strategy_id=19)
# print(submit_lake_backtest_result)

### Trade Report

In [20]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
1565,1.0,0.0,1.0,1.0,1.000000,615545.000000,0.000000,7.241955e+06,615545.000000
2640,1.0,0.0,1.0,1.0,1.000000,61500.000000,0.000000,4.735500e+06,61500.000000
1612,0.0,1.0,1.0,1.0,0.000000,0.000000,-34250.000000,4.740200e+06,-34250.000000
3038,0.0,1.0,1.0,1.0,0.000000,0.000000,-49200.000000,2.263200e+06,-49200.000000
2634,1.0,1.0,2.0,0.0,0.500000,62446.915772,-111752.814388,3.346310e+04,-49305.898616
1720,1.0,2.0,3.0,0.0,0.333333,8499.007291,-74691.544812,5.058434e+04,-66192.537521
6147,0.0,1.0,1.0,1.0,0.000000,0.000000,-74619.600000,4.318420e+06,-74619.600000
8016,1.0,3.0,4.0,0.0,0.250000,54180.411495,-180717.586826,6.204218e+04,-126537.175331
6691,1.0,1.0,2.0,1.0,0.500000,84020.778660,-231000.000000,4.532259e+06,-146979.221340


### Traded instruments

In [21]:
tidal.account.trades.keys()

dict_keys(['1301', '2640', '1720', '4915', '2014', '1612', '8016', '6691', '6146', '3010', '6290', '2474', '2634', '2457', '1565', '3357', '6409', '6147', '3038'])

### Trading History

In [22]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [23]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [24]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


[2024/11/28 10:18:38] werkzeug INFO WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:40393
 * Running on http://10.136.1.150:40393
[2024/11/28 10:18:38] werkzeug INFO Press CTRL+C to quit
[2024/11/28 10:18:46] werkzeug INFO 10.0.10.82 - - [28/Nov/2024 10:18:46] "GET / HTTP/1.1" 200 -
[2024/11/28 10:18:46] werkzeug INFO 10.0.10.82 - - [28/Nov/2024 10:18:46] "GET /static/js/main.d754b0a3.js HTTP/1.1" 200 -
[2024/11/28 10:18:47] werkzeug INFO 10.0.10.82 - - [28/Nov/2024 10:18:47] "GET /static/css/main.bf4d504b.css HTTP/1.1" 200 -
[2024/11/28 10:18:47] werkzeug INFO 10.0.10.82 - - [28/Nov/2024 10:18:47] "GET /images/Tidal_Logo_white.png HTTP/1.1" 200 -
[2024/11/28 10:18:47] werkzeug INFO 10.0.10.82 - - [28/Nov/2024 10:18:47] "GET /api/quote/inst_list HTTP/1.1" 200 -
[2024/11/28 10:18:47] werkzeug INFO 10.0.10.82 - - [28/Nov/2024 10:18:47] "GET /api/metr

In [ ]:
tidal.account.position_history

In [ ]:
Portfolio_info = tidal.metrics["Portfolio"].report
print(Portfolio_info)